# Download CFAR from dbGaP
**Author:** Jesse Marks <br>

## Download Aspera Connect
Aspera Connect is needed to download the data from dbGaP. We follow the directions at [this GitHub repo](https://gist.github.com/mfansler/71f09c8b6c9a95ec4e759a8ffc488be3).

In [ ]:
cd /shared/jmarks/bin

## Check for latest link: http://downloads.asperasoft.com/en/downloads/8?list
wget -qO- https://download.asperasoft.com/download/sw/connect/3.9.6/ibm-aspera-connect-3.9.6.173386-linux-g2.12-64.tar.gz |\
    tar xvz

## run it
chmod +x ibm-aspera-connect-3.9.6.173386-linux-g2.12-64.sh
./ibm-aspera-connect-3.9.6.173386-linux-g2.12-64.sh

## add it to the path now and in the future
export PATH=$PATH:~/.aspera/connect/bin/
echo 'export PATH=$PATH:~/.aspera/connect/bin/' >> ~/.bash_profile

## Download Data
After creating a dbGaP authorized access data request, execute the following in your shell to download the data.

In [ ]:
cd /shared/jmarks/proj/hiv/cfar

"$HOME/.aspera/connect/bin/ascp" -QTr -l 300M -k 1 -i \
    "$HOME/.aspera/connect/etc/asperaweb_id_dsa.openssh" -W \
    A294A459507974743BE3379E372FBA117A4CE5CB4F217445B839BCD4CBE62884A6E405F6DFDE644978CFA5455902F71A7D \
    dbtest@gap-upload.ncbi.nlm.nih.gov:data/instant/cloviseoj/70563 .

## Decrypt Data: sratoolkit
These are NCBI encrypted data and therefore need to be decyrpted. We use the sratoolkit to decrypt the data. A guide explaining the steps to download dbGaP data using the SRA toolkit can be found [here](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc&f=dbgap_use). In short, you need to 
* download the dbGaP repository key
* configure sratoolkit to point to this key
* decrypt the data

In [ ]:
## Download project key from dbGaP authorized access

## upload key to EC2 instance
scp -i ~/.ssh/gwas_rsa prj_2556.ngc   ec2-user@54.84.72.140:/shared/jmarks/ncbi/dbGaP-2556/

## configure sratoolkit
/shared/bioinformatics/software/third_party/sratoolkit.2.8.2-1-centos_linux64/bin/vdb-config -i

## In order to access the projects data, you will need to change directory or "cd" to the project's workspace.
cd /shared/jmarks/ncbi/dbGaP-2556/
/shared/bioinformatics/software/third_party/sratoolkit.2.8.2-1-centos_linux64/bin/vdb-decrypt -q \
    /shared/jmarks/proj/hiv/cfar/70563 &


## Upload to S3

In [ ]:

cd /shared/jmarks/proj/hiv/cfar/70563/PhenoGenotypeFiles/
aws s3 sync . s3://rti-common/dbGaP/phs001788_cfar/